# 📊 Workshop Attendance Analysis (Notebook Version)

This notebook performs the attendance analysis. 

**Instructions:**
1.  **Insert Data Paths**: Update the file paths in the code cell below.
2.  **Run All Cells**: Execute the cells to load data and generate graphs.
3.  **Check Attributes**: The "Data Overview" section will display all available columns (attributes) for your reference.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import numpy as np

# -----------------------------------------
# 1️⃣ Setup & Configuration
# -----------------------------------------
sns.set_style("whitegrid")
pd.set_option('display.max_columns', None)

# 👇 INSERT YOUR FILE PATHS HERE 👇
attendance_file_paths = [
    '/content/2023_2024_2025__Attendance_Report (1).xlsx',
    # '/content/Another_File.csv', 
]
taxonomy_file_path = '/content/Taxonomy.csv'

In [ ]:
# -----------------------------------------
# 2️⃣ Load & Inspect Data (Interface)
# -----------------------------------------
def load_and_display_data():
    # --- Load Attendance ---
    df_list = []
    for path in attendance_file_paths:
        if os.path.exists(path):
            try:
                if path.lower().endswith('.csv'):
                    df_temp = pd.read_csv(path)
                else:
                    df_temp = pd.read_excel(path)
                df_list.append(df_temp)
                print(f"✅ Loaded: {path}")
            except Exception as e:
                print(f"❌ Error loading {path}: {e}")
        else:
            print(f"⚠️ File not found: {path}")

    if df_list:
        df_att = pd.concat(df_list, ignore_index=True)
    else:
        print("❌ No valid attendance data found.")
        return pd.DataFrame(), pd.DataFrame()

    # --- Load Taxonomy ---
    if os.path.exists(taxonomy_file_path):
        try:
            if taxonomy_file_path.lower().endswith('.csv'):
                df_tax = pd.read_csv(taxonomy_file_path)
            else:
                df_tax = pd.read_excel(taxonomy_file_path)
            print(f"✅ Loaded: {taxonomy_file_path}")
        except Exception as e:
            print(f"❌ Error loading taxonomy: {e}")
            df_tax = pd.DataFrame()
    else:
        print(f"⚠️ Taxonomy file not found: {taxonomy_file_path}")
        df_tax = pd.DataFrame()

    return df_att, df_tax

df, df_taxonomy = load_and_display_data()

if not df.empty:
    print("\n" + "="*40)
    print("📋 DATA INTERFACE: AVAILABLE ATTRIBUTES")
    print("="*40)
    print(f"Total Records: {len(df)}")
    print("\n--- Attendance Data Columns ---")
    for col in df.columns:
        print(f" • {col}")
    
    print("\n--- Taxonomy Data Columns ---")
    if not df_taxonomy.empty:
        for col in df_taxonomy.columns:
            print(f" • {col}")
    else:
        print(" (No Taxonomy Data Loaded)")
    print("="*40 + "\n")
else:
    print("❌ STOP: Please fix file paths above.")

In [ ]:
# -----------------------------------------
# 3️⃣ Data Cleaning & Merging
# -----------------------------------------
if not df.empty and not df_taxonomy.empty:
    # Clean Columns
    df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('[^a-z0-9_]', '', regex=True)
    
    rename_map = {
        'simid': 'student_id', 
        'university_program': 'university_program',
        'attendance_status': 'attendance_status',
        'event_name': 'event_name',
        'attended_date': 'attended_date',
        'attended_time': 'attended_time',
        'registered_date': 'registered_date',
        'citizenship': 'citizenship',
        'nationality': 'nationality',
        'expected_grad_term': 'expected_grad_term'
    }
    df = df.rename(columns=rename_map)

    # Clean Event Names
    if 'event_name' in df.columns:
        df['event_name_clean'] = df['event_name'].str.replace(r'[\(\)]', '', regex=True).str.strip().str.lower()
    
    if 'Career Development Workshop Titles' in df_taxonomy.columns:
        df_taxonomy['workshop_title_clean'] = df_taxonomy['Career Development Workshop Titles'].str.replace(r'[\(\)]', '', regex=True).str.strip().str.lower()

    # Merge
    df = df.merge(
        df_taxonomy[['workshop_title_clean', 'Category', 'Sub-Category']], 
        left_on='event_name_clean', 
        right_on='workshop_title_clean', 
        how='left'
    )

    df['Category'] = df['Category'].fillna('Uncategorized')
    df['Sub-Category'] = df['Sub-Category'].fillna('Uncategorized')
    df['is_attended'] = df['attendance_status'].apply(lambda x: 1 if str(x).lower() == 'attended' else 0)
    
    # University
    if 'university_program' not in df.columns:
        possible_cols = [c for c in df.columns if 'university' in c or 'program' in c]
        if possible_cols:
            df['university_program'] = df[possible_cols[0]]
        else:
            df['university_program'] = 'Unknown'
    
    df['uni_category'] = df['university_program'].fillna('Unknown')
    df_attended = df[df['is_attended'] == 1].copy()
    
    print("✅ Data Merged and Cleaned.")

## 1. Overall Attendance Overview
Analyze total attendance per student (single vs. multi-run)

In [ ]:
if not df_attended.empty:
    student_counts = df_attended.groupby('student_id')['event_name'].count()
    single_attendees = student_counts[student_counts == 1].count()
    multi_attendees = student_counts[student_counts > 1].count()
    
    plt.figure(figsize=(8, 6))
    plt.pie([single_attendees, multi_attendees], labels=['Single Workshop', 'Multiple Workshops'], 
            autopct='%1.1f%%', colors=['#66b3ff', '#ff9999'], startangle=90)
    plt.title('Student Attendance: Single vs Multi-Run', fontsize=14, fontweight='bold')
    plt.show()

## 2. Unique Participant Count
Count total unique attendees across all workshops

In [ ]:
if not df_attended.empty:
    unique_count = df_attended['student_id'].nunique()
    print(f"\n🏆 Total Unique Participants: {unique_count}\n")

## 3. Most Popular Days & Time Slots
Identify days/times with highest attendance

In [ ]:
if not df_attended.empty and 'attended_date' in df_attended.columns:
    # Convert to datetime
    df_attended['attended_date_dt'] = pd.to_datetime(df_attended['attended_date'], errors='coerce')
    
    # Day of Week
    day_counts = df_attended['attended_date_dt'].dt.day_name().value_counts()
    # Sort by normal week order
    days_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    day_counts = day_counts.reindex(days_order).dropna()
    
    plt.figure(figsize=(10, 5))
    sns.barplot(x=day_counts.index, y=day_counts.values, palette='viridis')
    plt.title('Attendance by Day of Week', fontsize=14)
    plt.ylabel('Attendees')
    plt.show()

if not df_attended.empty and 'attended_time' in df_attended.columns:
    # Try to extract hour
    # Assuming time format might vary, let's try to convert
    # If it's a string like '14:00:00', pd.to_datetime works
    try:
        df_attended['hour'] = pd.to_datetime(df_attended['attended_time'].astype(str), format='%H:%M:%S', errors='coerce').dt.hour
        # Fallback for other formats if needed
        if df_attended['hour'].isnull().all():
             df_attended['hour'] = pd.to_datetime(df_attended['attended_time'].astype(str), errors='coerce').dt.hour

        hour_counts = df_attended['hour'].value_counts().sort_index()
        
        plt.figure(figsize=(10, 5))
        sns.barplot(x=hour_counts.index, y=hour_counts.values, color='teal')
        plt.title('Attendance by Hour of Day', fontsize=14)
        plt.xlabel('Hour (24h)')
        plt.ylabel('Attendees')
        plt.show()
    except Exception as e:
        print(f"Could not parse time: {e}")

## 4. Attendance by University
Examine attendance distribution across universities

In [ ]:
if not df_attended.empty:
    uni_counts = df_attended['uni_category'].value_counts()
    
    plt.figure(figsize=(12, 6))
    sns.barplot(y=uni_counts.index, x=uni_counts.values, palette='magma')
    plt.title('Total Attendance by University Program', fontsize=14)
    plt.xlabel('Count')
    plt.show()

## 5. Workshop Attendance by University & Subcategory
Evaluate attendance variations by workshop subcategory per university, and display top 10 workshops title in each category in horizontal stack bar grapgh in ascending order within the a bar graph

In [ ]:
def plot_stacked_bar(df_data, title):
    if df_data.empty:
        return

    # Pivot
    df_pivot = df_data.pivot(index='event_name', columns='uni_category', values='is_attended').fillna(0)
    
    # Sort Rows (Total Attendance Ascending -> Largest at Bottom/Top depending on plot)
    df_pivot['total'] = df_pivot.sum(axis=1)
    df_pivot = df_pivot.sort_values(by='total', ascending=True) 
    df_pivot = df_pivot.drop(columns=['total'])

    # Sort Columns (Largest Uni on Left)
    uni_totals = df_pivot.sum(axis=0).sort_values(ascending=False)
    df_pivot = df_pivot[uni_totals.index.tolist()]

    # Plot
    plt.figure(figsize=(16, max(6, len(df_pivot) * 0.6)))
    all_universities = sorted(df_attended['uni_category'].unique())
    color_palette = plt.cm.tab20.colors
    university_colors = {uni: color_palette[i % len(color_palette)] for i, uni in enumerate(all_universities)}
    colors = [university_colors.get(col, '#333333') for col in df_pivot.columns]
    
    ax = df_pivot.plot(kind='barh', stacked=True, color=colors, width=0.7, ax=plt.gca())
    
    plt.title(title, fontsize=16, weight='bold')
    plt.xlabel('Total Attendance', fontsize=12)
    plt.ylabel('Workshop Title', fontsize=12)
    
    # Numbers
    max_val = df_pivot.sum(axis=1).max()
    plt.xlim(0, max_val * 1.25)
    for i, event in enumerate(df_pivot.index):
        cumulative = 0
        for col in df_pivot.columns:
            val = df_pivot.loc[event, col]
            if val > 0:
                ax.text(cumulative + val/2, i, f'{int(val)}', 
                        ha='center', va='center', color='white', fontsize=9, fontweight='bold')
                cumulative += val
        ax.text(cumulative + max_val*0.01, i, f'Total: {int(cumulative)}', 
                ha='left', va='center', color='black', fontweight='bold')

    plt.legend(title='University Program', bbox_to_anchor=(1.02, 1), loc='upper left')
    plt.tight_layout()
    plt.show()

if not df_attended.empty:
    categories = df_attended['Category'].unique()
    for cat in categories:
        sub_cats = df_attended[df_attended['Category'] == cat]['Sub-Category'].unique()
        for sub in sub_cats:
            df_sub = df_attended[(df_attended['Category'] == cat) & (df_attended['Sub-Category'] == sub)]
            df_agg = df_sub.groupby(['event_name', 'uni_category'])['is_attended'].sum().reset_index()
            event_totals = df_agg.groupby('event_name')['is_attended'].sum().reset_index()
            top_10_events = event_totals.sort_values('is_attended', ascending=False).head(10)['event_name'].tolist()
            df_plot = df_agg[df_agg['event_name'].isin(top_10_events)]
            
            plot_stacked_bar(df_plot, f'Top 10 Workshops: {cat} - {sub}')

## 6. Attendance by Student Type (Local vs. International)
Compare attendance trends

In [ ]:
if not df_attended.empty and ('citizenship' in df_attended.columns or 'nationality' in df_attended.columns):
    # Determine Local vs International
    # Priority: Citizenship -> Nationality
    col_to_use = 'citizenship' if 'citizenship' in df_attended.columns else 'nationality'
    
    def get_student_type(val):
        val_str = str(val).lower()
        if 'singapore' in val_str or 'pr' in val_str or 'permanent resident' in val_str:
            return 'Local'
        return 'International'

    df_attended['student_type'] = df_attended[col_to_use].apply(get_student_type)
    
    type_counts = df_attended['student_type'].value_counts()
    
    plt.figure(figsize=(8, 6))
    plt.pie(type_counts.values, labels=type_counts.index, autopct='%1.1f%%', colors=['#ffcc99', '#99ff99'], startangle=90)
    plt.title(f'Attendance by Student Type (Based on {col_to_use})', fontsize=14, fontweight='bold')
    plt.show()

## 7. Attendance by Expected Graduation Period
Assess patterns based on graduation year

In [ ]:
if not df_attended.empty and 'expected_grad_term' in df_attended.columns:
    grad_counts = df_attended['expected_grad_term'].value_counts().sort_index()
    
    plt.figure(figsize=(12, 6))
    sns.barplot(x=grad_counts.index, y=grad_counts.values, palette='coolwarm')
    plt.title('Attendance by Expected Graduation Term', fontsize=14)
    plt.xticks(rotation=45)
    plt.show()

## 8. Attendance Attribution Based on Registration Timing
Analyze early vs. late registration and actual attendance

In [ ]:
if not df_attended.empty and 'registered_date' in df_attended.columns and 'attended_date' in df_attended.columns:
    df_attended['reg_dt'] = pd.to_datetime(df_attended['registered_date'], errors='coerce')
    df_attended['att_dt'] = pd.to_datetime(df_attended['attended_date'], errors='coerce')
    
    # Calculate days in advance
    df_attended['days_advance'] = (df_attended['att_dt'] - df_attended['reg_dt']).dt.days
    
    def categorize_registration(days):
        if pd.isna(days):
            return 'Unknown'
        if days >= 14:
            return 'Early Bird (>2 weeks)'
        elif days >= 3:
            return 'Standard (3-14 days)'
        else:
            return 'Last Minute (<3 days)'

    df_attended['reg_category'] = df_attended['days_advance'].apply(categorize_registration)
    
    reg_counts = df_attended['reg_category'].value_counts()
    # Sort order
    order = ['Early Bird (>2 weeks)', 'Standard (3-14 days)', 'Last Minute (<3 days)', 'Unknown']
    reg_counts = reg_counts.reindex(order).dropna()
    
    plt.figure(figsize=(10, 6))
    sns.barplot(x=reg_counts.index, y=reg_counts.values, palette='Blues_d')
    plt.title('Attendance Attribution by Registration Timing', fontsize=14)
    plt.ylabel('Attendees')
    plt.show()